In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn import feature_extraction, linear_model, model_selection, preprocessing, metrics
import string
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, classification
import time
import random

In [ ]:
import pandas as pd
sample_submission = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
test = pd.read_csv("../input/nlp-getting-started/test.csv")
train = pd.read_csv("../input/nlp-getting-started/train.csv")

In [ ]:
def model_evaluator(model,train_data,target_data,test_data):
    model.fit(train_data,target_data)
    score=model_selection.cross_val_score(model,train_data,target_data,cv=3,scoring='f1')
    return score

In [ ]:
#exploring data
train.head()

### COUNT VECTORIZER

In [ ]:
#count vectorizing string-data
count_vectorizer=feature_extraction.text.CountVectorizer()
train_vectors=count_vectorizer.fit_transform(train.text[:])
test_vectors=count_vectorizer.transform(test.text[:])
target_vectors=train.target[:]

In [ ]:
diff=train_vectors.mean(axis=0)-test_vectors.mean(axis=0)
sns.scatterplot(x=np.arange(diff.shape[1]),y=np.array(diff)[0])

In [ ]:
#evaluating val score of count_vectrized Ridge_data
clf=linear_model.RidgeClassifier()
clf.fit(train_vectors,target_vectors)
score=model_selection.cross_val_score(clf,train_vectors,target_vectors,cv=5,scoring='f1')
a=score
print('cross validation score for count_vectorized data:',a.mean())

In [ ]:
#predicting for test data
clf.fit(train_vectors,target_vectors)
test_preds=clf.predict(test_vectors)

#making final submission
a=pd.DataFrame({'id':test.id,'target':test_preds})
#a.to_csv('submission.csv',index=False)

### COUNT VECTORIZER and tf-idf transformation

In [ ]:
# tf-idf processing the data
tfidf=feature_extraction.text.TfidfTransformer()
train_tfidf=tfidf.fit_transform(train_vectors)
test_tfidf=tfidf.transform(test_vectors)
clf=linear_model.RidgeClassifier()

In [ ]:
train_tfidf.todense().shape

In [ ]:
#evaluation tf_idf data with Ridge classifier (score improvement is observed)
clf=linear_model.RidgeClassifier()
clf.fit(train_tfidf,target_vectors)
score=model_selection.cross_val_score(clf,train_tfidf,target_vectors,cv=5,scoring='f1')
a=score
print('cross validation score for count_vectorized data:',a.mean())

In [ ]:
#predicting for test data
clf.fit(train_tfidf,target_vectors)
test_preds=clf.predict(test_tfidf)

#making final submission
a=pd.DataFrame({'id':test.id,'target':test_preds})
#a.to_csv('submission.csv',index=False)

### TEXT CLEANING, COUNT VECTORIZER, TF IDF

In [ ]:
def text_cleaning(text):
    a=[char for char in text if char not in string.punctuation]
    a=''.join(a)
    a=a.split()
    #a=[c for c in a if c.lower() not in stopwords.words('english')]
    return  a

In [ ]:
#text cleaning done
train['text'].apply(text_cleaning)

In [ ]:
count_vectorizer=feature_extraction.text.CountVectorizer(analyzer=text_cleaning).fit(train['text'])
clf=linear_model.RidgeClassifier()

In [ ]:
#vectorizing and tf_idf transforming again and evaluating
train_vectors=count_vectorizer.transform(train.text)
test_vectors=count_vectorizer.transform(test.text)
target_vectors=train.target[:]
tfidf=feature_extraction.text.TfidfTransformer()
train_tfidf=tfidf.fit_transform(train_vectors)
test_tfidf=tfidf.transform(test_vectors)

In [ ]:
X_train, X_val, y_train, y_val =train_test_split(train_tfidf, target_vectors, test_size=0.33, random_state=42)

In [ ]:
clf=linear_model.RidgeClassifier()
clf.fit(X_train,y_train)

In [ ]:
accuracy=[]
f1_set=[]
for i in np.arange(-0.5,1.5,step=0.1):
    temp=pd.DataFrame(clf.decision_function(X_val))[0].apply(lambda x: 1 if x>i else 0)
    accuracy.append((temp.values==y_val.values).sum())
    
    tn, fp, fn, tp =confusion_matrix(y_val,temp).ravel()
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    f1=(2*precision*recall)/(precision+recall)  
    f1_set.append(f1)
accuracy=np.array(accuracy)
f1_set=np.array(f1_set)

In [ ]:
plt.scatter(np.arange(len(f1_set))*0.1-0.5,f1_set)

In [ ]:
f1_set[f1_set.argmax()]

In [ ]:
temp=pd.DataFrame(clf.decision_function(X_val))[0].apply(lambda x: 1 if x>-0 else 0)
print((temp.values==y_val.values).sum())

In [ ]:
tn, fp, fn, tp =confusion_matrix(y_val,temp).ravel()
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=(2*precision*recall)/(precision+recall)
print(f1)

In [ ]:
#predicting for test data
clf.fit(train_tfidf,target_vectors)
test_preds=pd.DataFrame(clf.decision_function(test_tfidf))[0].apply(lambda x: 1 if x>-0.11 else 0)

#making final submission
a=pd.DataFrame({'id':test.id,'target':test_preds})
a.to_csv('submission.csv',index=False)

In [ ]:
target_vectors.mean()

In [ ]:
test_preds.mean()

# LOGISTIC REGRESSION WITH L2 penalty

In [ ]:
clf=linear_model.LogisticRegression(penalty='l2')
clf.fit(X_train,y_train)

In [ ]:
sns.distplot(pd.DataFrame(clf.predict_proba(X_val))[1])

In [ ]:
pd.DataFrame(clf.predict_proba(X_val))[1].apply(lambda x: 1 if x>0.42 else 0).sum()

In [ ]:
y_val.sum()

In [ ]:
test_preds=pd.DataFrame(clf.predict_proba(test_tfidf))[1].apply(lambda x: 1 if x>0.42 else 0)

#making final submission
a=pd.DataFrame({'id':test.id,'target':test_preds})
a.to_csv('submission.csv',index=False)

In [ ]:
test

# ANN

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
input_shape = [X_train.shape[1]]

model = keras.Sequential([layers.BatchNormalization(),
                         layers.Dense(256,activation='relu',input_shape=input_shape),
                         layers.BatchNormalization(),
                         layers.Dropout(0.3),
                         layers.Dense(1,activation='sigmoid')])

In [ ]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['binary_accuracy'])

In [ ]:
X_train.toarray()

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)

history = model.fit(
    X_train.toarray(), y_train.values,
    validation_data=(X_val.toarray(), y_val.values),
    batch_size=512,
    epochs=200,
    callbacks=[early_stopping])

history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot(title="Cross-entropy")
history_df.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot(title="Accuracy")

In [ ]:
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

In [ ]:
t=model.predict_proba(X_val.toarray())
sns.distplot(pd.DataFrame(t))

In [ ]:
t_test=pd.DataFrame(model.predict_proba(test_tfidf.toarray()))
sns.distplot(pd.DataFrame(t_test))

In [ ]:
test_preds=pd.DataFrame(model.predict_proba(test_tfidf.toarray()))[0].apply(lambda x: 1 if x>0.7 else 0)
a=pd.DataFrame({'id':test.id,'target':test_preds})
a.to_csv('submission.csv',index=False)

# Decision Trees

In [ ]:
clf=tree.DecisionTreeClassifier(max_depth=180)
clf.fit(X_train,y_train)
score=model_selection.cross_val_score(clf,X_train,y_train,cv=5,scoring='f1')
a=score
print('cross validation score for count_vectorized data:',a.mean())

In [ ]:
y_pred=clf.predict_proba(X_val)
predicted_probs=pd.DataFrame(y_pred)
predicted_probs[2]=predicted_probs[0].apply(lambda x: 0 if x>0.5 else 1)
y_prednew=predicted_probs[2].values
tn, fp, fn, tp =confusion_matrix(y_val,y_prednew).ravel()
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=(2*precision*recall)/(precision+recall)
print("f1= ",f1)

In [ ]:
sns.distplot(pd.DataFrame(y_pred)[1])

In [ ]:
#predicting for test data
clf.fit(train_tfidf,target_vectors)
test_preds=clf.predict(test_tfidf)

#making final submission
a=pd.DataFrame({'id':test.id,'target':test_preds})
#a.to_csv('submission.csv',index=False)

In [ ]:
clf.get_depth()

In [ ]:
%%time
err=[]
for i in np.arange(5,411,step=5): 
    clf=tree.DecisionTreeClassifier(max_depth=i)
    clf.fit(train_tfidf,target_vectors)
    score=model_selection.cross_val_score(clf,train_tfidf,target_vectors,cv=5,scoring='f1')
    a=score
    print('cross validation score for count_vectorized data:',a.mean())
    err.append(a.mean())

err=np.array(err)
plt.scatter(x=range(len(err)),y=err)

In [ ]:
plt.scatter(x=np.arange(len(err))*5+5,y=err,)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_tfidf,target_vectors, test_size=0.3, random_state=0)

In [ ]:
clf=RandomForestClassifier(max_features="sqrt",n_estimators=400,oob_score=False,random_state=0)
clf.fit(X_train,y_train)

In [ ]:
err=[]
for i in range(400):
    y_pred=clf.estimators_[i].predict(X_val)
    tn, fp, fn, tp =confusion_matrix(y_val,y_pred).ravel()
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    f1=(2*precision*recall)/(precision+recall)
    err.append(f1)
    
err=np.array(err)

In [ ]:
cum_err=[]
for i in range(len(err)):
    temp=sum(err[0:i]/(i+1))
    cum_err.append(temp)
    
cum_err=np.array(cum_err)

In [ ]:
plt.scatter(x=np.arange(400)+1,y=cum_err)

100 trees seem enough

In [ ]:
clf=RandomForestClassifier(max_features="auto",n_estimators=40,oob_score=False,random_state=0)
clf.fit(X_train,y_train)

In [ ]:
err=[]
for i in range(40):
    y_pred=clf.estimators_[i].predict(X_val)
    tn, fp, fn, tp =confusion_matrix(y_val,y_pred).ravel()
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    f1=(2*precision*recall)/(precision+recall)
    err.append(f1)
    
err=np.array(err)

cum_err=[]
for i in range(len(err)):
    temp=sum(err[0:i]/(i+1))
    cum_err.append(temp)
    
cum_err=np.array(cum_err)

plt.scatter(x=np.arange(40)+1,y=cum_err)
plt.xlabel('number of trees')
plt.ylabel('f1-score')

In [ ]:
depth=[]
for i in range(len(clf.estimators_)):
    depth.append(clf.estimators_[0].get_depth())
    
depth=np.array(depth)

In [ ]:
err=[]
for i in np.arange(1,180):
    clf=RandomForestClassifier(max_depth=i,n_estimators=40,oob_score=False,random_state=0)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_val)
    tn, fp, fn, tp =confusion_matrix(y_val,y_pred).ravel()
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    f1=(2*precision*recall)/(precision+recall)
    err.append(f1)
    
err=np.array(err)

In [ ]:
plt.scatter(x=np.arange(len(err))+1,y=err)
plt.xlabel('max_depth')
plt.ylabel('f1-score')

max depth of 100 seems fine

In [ ]:
err=[]
for i in np.arange(1000,26000,step=1000):
    clf=RandomForestClassifier(max_depth=100,n_estimators=40,max_features=i,random_state=0)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_val)
    tn, fp, fn, tp =confusion_matrix(y_val,y_pred).ravel()
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    f1=(2*precision*recall)/(precision+recall)
    err.append(f1)
    
err=np.array(err)

In [ ]:
plt.scatter(x=np.arange(len(err))+1,y=err)
plt.xlabel('max_features')
plt.ylabel('f1-score')

In [ ]:
err.argmax()

3000 seems fine

In [ ]:
ts=time.time()
clf=RandomForestClassifier(max_features="auto",n_estimators=150,oob_score=False,random_state=0)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_val)
tn, fp, fn, tp =confusion_matrix(y_val,y_pred).ravel()
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=(2*precision*recall)/(precision+recall)
print("f1= ",f1)
print("time taken", time.time()-ts)

In [ ]:
clf.estimators_[3].get_depth()

In [ ]:
f1_set=[]
for i in np.arange(0.3,0.9,step=0.01):
    y_pred=clf.predict_proba(X_val)
    predicted_probs=pd.DataFrame(y_pred)
    predicted_probs[2]=predicted_probs[0].apply(lambda x: 0 if x>i else 1)
    y_prednew=predicted_probs[2].values
    tn, fp, fn, tp =confusion_matrix(y_val,y_prednew).ravel()
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    f1=(2*precision*recall)/(precision+recall)
    f1_set.append(f1)
    
f1_set=np.array(f1_set)

In [ ]:
plt.scatter(np.arange(0.3,0.9,step=0.01),f1_set)

In [ ]:
thres=f1_set.argmax()*0.01+0.3

In [ ]:
thres

RF final model

In [ ]:
clf=RandomForestClassifier(max_features="sqrt",n_estimators=150,oob_score=False,random_state=0)
clf.fit(train_tfidf,target_vectors)
test_preds=clf.predict_proba(test_tfidf)
predicted_probs=pd.DataFrame(test_preds)
predicted_probs[2]=predicted_probs[0].apply(lambda x: 0 if x>0.62 else 1)
y_prednew=predicted_probs[2].values
#making final submission
a=pd.DataFrame({'id':test.id,'target':y_prednew})
a.to_csv('submission.csv',index=False)

In [ ]:
predicted_probs[2]=predicted_probs[0].apply(lambda x: 0 if x>0.62 else 1)
y_prednew=predicted_probs[2].values
y_prednew.mean()

In [ ]:
len(clf.estimators_)

# Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
random.seed(0) 

In [ ]:
ts=time.time()
clf = GradientBoostingClassifier(n_estimators=600, learning_rate=0.05
                                 max_depth=3,max_features="sqrt",random_state=0).fit(X_train, y_train)

print("time taken=",time.time()-ts)

In [ ]:
f1_sets=[]
for i in np.arange(0.3,0.9,step=0.01):
    y_pred=clf.predict_proba(X_val)
    predicted_probs=pd.DataFrame(y_pred)
    predicted_probs[2]=predicted_probs[0].apply(lambda x: 0 if x>i else 1)
    y_prednew=predicted_probs[2].values
    tn, fp, fn, tp =confusion_matrix(y_val,y_prednew).ravel()
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    f1=(2*precision*recall)/(precision+recall)
    f1_sets.append(f1)

f1_sets=np.array(f1_sets)

In [ ]:
thres=f1_sets.argmax()*0.01+0.3

In [ ]:
plt.scatter(np.arange(0.3,0.9,step=0.01),f1_sets)

In [ ]:
thres

In [ ]:
y_pred=clf.predict_proba(X_val)
predicted_probs=pd.DataFrame(y_pred)
predicted_probs[2]=predicted_probs[0].apply(lambda x: 0 if x>thres else 1)
y_prednew=predicted_probs[2].values
tn, fp, fn, tp =confusion_matrix(y_val,y_prednew).ravel()
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=(2*precision*recall)/(precision+recall)
print("f1=",f1)

In [ ]:
ts=time.time()
clf = GradientBoostingClassifier(n_estimators=600, learning_rate=1.0,
                                 max_depth=1, random_state=0).fit(train_tfidf, target_vectors)

print("time taken=",time.time()-ts)

y_pred=clf.predict_proba(test_tfidf)
predicted_probs=pd.DataFrame(y_pred)
predicted_probs[2]=predicted_probs[0].apply(lambda x: 0 if x>thres else 1)
y_prednew=predicted_probs[2].values
a=pd.DataFrame({'id':test.id,'target':y_prednew})
a.to_csv('submission.csv',index=False)

In [ ]:
ts=time.time()
f1_sets=[]
for  i in np.arange(2000,10000,step=100):
    clf = GradientBoostingClassifier(n_estimators=i, learning_rate=0.05,
                                 max_depth=3,max_features="sqrt", random_state=0).fit(X_train, y_train)
    y_pred=clf.predict_proba(X_val)
    predicted_probs=pd.DataFrame(y_pred)
    predicted_probs[2]=predicted_probs[0].apply(lambda x: 0 if x>thres else 1)
    y_prednew=predicted_probs[2].values
    tn, fp, fn, tp =confusion_matrix(y_val,y_prednew).ravel()
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    f1=(2*precision*recall)/(precision+recall)
    f1_sets.append(f1)
    print("number of estimators of last run=",i)
    print("time taken=",time.time()-ts)
    
f1_sets=np.array(f1_sets)

In [ ]:
plt.scatter(np.arange(2000,5100,step=100),f1_sets)

In [ ]:
print("mean of training targets",y_train.mean())
print("mean of validation targets",y_val.mean())
print()